In [158]:
# from chess import Piece
# from chess.Board import Board
import numpy as np
import torch
import torch.nn.functional as F
import math


In [202]:
class Piece:
  def __init__(self, type = None, color = None):
    self.color = color
    if type is not None: self.type = type
    if self.color == 'b': self.type *= -1

  def vectorize(self):
    return F.one_hot(torch.tensor(self.type), num_classes=7)
  
  def render(self):
    if self.type == 0: return '_'
    assert(self.color is not None)
    chars = {
      'w': {
        1: '♙',
        2: '♘',
        3: '♗',
        4: '♖',
        # 5: '♖',
        5: '♕',
        6: '♔'
      }, 'b': {
        1: '♟',
        2: '♞',
        3: '♝',
        4: '♜',
        5: '♛',
        6: '♚'
      }, 
    }

    # return 'x'
    
    return chars[self.color][self.type]

  def from_type(type):
    color = None 
    # print(type)
    if type < 0: color = 'b'
    if type > 0: color = 'w'
    return Piece(type=abs(type), color=color)
    
  def Empty(color=None): return Piece(type = 0, color = color)
  def Pawn(color=None): return Piece(type = 1, color = color)
  def Knight(color=None): return Piece(type = 2, color = color)
  def Bishop(color=None): return Piece(type = 3, color = color)
  def Rook(color=None): return Piece(type = 4, color = color)
  def Queen(color=None): return Piece(type = 5, color = color)
  def King(color=None): return Piece(type = 6, color = color)

Piece.Empty().vectorize()
Piece.Empty().vectorize()


tensor([1, 0, 0, 0, 0, 0, 0])

In [203]:
class Board:
  def __init__(self):
    
    # self.squares = torch.tensor(np.array([] * 64))
    # self.squares = torch.tensor(
    # self.squares = np.full(64, [Piece.Empty().vectorize() for _ in range(64)])

    self.squares = torch.tensor([Piece.Empty('w').type for _ in range(64)]).view(8, 8)
    # self.squares = torch.tensor(self.squares)
# .fill(Piece.Empty().vectorize())
    # self.squares = torch.tensor(np.zeros((8, 8))) # create an 8 * 8 grid
    self.build()
  
  def render(self):
    out = ""
    for row in self.squares:
      for col in row:
        # out += " ♞ "
        piece = Piece.from_type(col.item())
        out += f" {piece.render()} "
      out += "\n"
    return out
        
  def build(self):
    self.place(1, 0, Piece.Pawn('w').type)
    
  def place(self, x, y, piece):
    self.squares[x][y] = piece
  
  def __repr__(self):
    return self.render()


In [204]:
board = Board()
board

 _  _  _  _  _  _  _  _ 
 ♙  _  _  _  _  _  _  _ 
 _  _  _  _  _  _  _  _ 
 _  _  _  _  _  _  _  _ 
 _  _  _  _  _  _  _  _ 
 _  _  _  _  _  _  _  _ 
 _  _  _  _  _  _  _  _ 
 _  _  _  _  _  _  _  _ 